#### Using column_property

The `column_property()` function can be used to __map a SQL expression__ in a manner similar to a _regularly mapped_ `Column`. With this technique, the _attribute_ is __loaded along with all other column-mapped attributes at load time__. This is in some cases an _advantage_ over the usage of hybrids, as the value can be loaded up front at the same time as the parent row of the object, particularly if the expression is one which links to other tables (typically as a _correlated subquery_) to access data that wouldn't normally be available on an already loaded object.

_Disadvantages_ to using `column_property()` for SQL expressions include that the __expression must be compatible with the SELECT statement emitted__ for the class as a whole, and there are also _some configurational quirks_ which can occur when using `column_property()` from _declarative mixins_.

In [1]:
from sqlalchemy import (
    Column, ForeignKey, Integer, String, Table,
    select, func, inspect, and_,
)
from sqlalchemy.orm import column_property, registry
from sqlalchemy.ext.declarative import declarative_base

In [2]:
Base = declarative_base()
reg = registry()

In [3]:
class SimpleUser(Base):
    __tablename__ = "simple_user"
    
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))
    fullname = column_property(firstname + " " + lastname)

_Correlated subqueries_ may be used as well. Below we use the `select()` construct to create a `ScalarSelect`, representing a __column-oriented `SELECT` statement__, that _links together the count_ of `Address` objects available for a particular `User`.

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("user.id"))

In [5]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    address_count = column_property(
        select(func.count(Address.id)).
        where(Address.user_id == id).
        correlate_except(Address).
        scalar_subquery()
    )

In the above example, we define a `ScalarSelect()` construct like the following:

```
stmt = (
    select(func.count(Address.id))
    .where(Address.user_id == id)
    .correlate_except(Address)
    .scalar_subquery()
)
```

Above, we first use `select()` to create a `Select` construct, which we then __convert into a scalar subquery__ using the `Select.scalar_subquery()` method, indicating our intent to use this `Select` statement in a _column expression context_.

Within the `Select` itself, we select the _count_ of `Address.id` rows where the `Address.user_id` column is __equated to id__, which in the context of the `User` class is the `Column` named _id_ (note that _id_ is also the name of a Python built in function, which is not what we want to use here - if we were outside of the `User` class definition, we'd use User.id).

The `Select.correlate_except()` method indicates that each element in the `FROM` clause of this `select()` __may be omitted from the `FROM` list__ (that is, _correlated_ to the __enclosing `SELECT` statement__ against `User`) except for the one corresponding to `Address`. This __isn't strictly necessary__, but _prevents_ `Address` from __being inadvertently omitted__ from the `FROM` list in the case of a _long string of joins_ between `User` and `Address` tables where `SELECT` statements against `Address` are _nested_.

If import issues prevent the `column_property()` from being _defined inline_ with the class, it __can be assigned to the class after both are configured__. When _using mappings_ that make use of a `declarative_base()` base class, this _attribute assignment_ has the effect of calling `Mapper.add_property()` to add an additional property after the fact.

```
# only works if a declarative base class is in use
User.address_count = column_property(
    select(func.count(Address.id)).where(Address.user_id == User.id).scalar_subquery()
)
```

When using _mapping styles_ that __don't use__ `declarative_base()`, such as the `registry.mapped()` decorator, the `Mapper.add_property()` method __may be invoked explicitly__ on the underlying `Mapper` object, which can be obtained using `inspect()`.

In [6]:
class MappedAddress(Base):
    __tablename__ = "mapped_address"
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("mapped_user.id"))

In [7]:
@reg.mapped
class MappedUser:
    __tablename__ = "mapped_user"
    
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))

In [8]:
inspect(MappedUser).add_property(
    "address_count", column_property(
        select(func.count(MappedAddress.id)).
        where(MappedAddress.user_id == MappedUser.id).
        scalar_subquery()
    )
)

For a `column_property()` that refers to columns linked from a _many-to-many relationship_, use `and_()` to __join the fields of the association table__ to both tables in a _relationship_.

In [9]:
books = Table(
    "books",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [10]:
authors = Table(
    "authors",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [11]:
book_authors = Table(
    "book_authors",
    Base.metadata,
    Column("author_id", Integer, ForeignKey("authors.id")),
    Column("book_id", Integer, ForeignKey("books.id")),
)

In [12]:
class Author(Base):
    __table__ = authors
    
    book_count = column_property(
        select(func.count(books.c.id)).
        where(
            and_(
                book_authors.c.author_id == authors.c.id,
                book_authors.c.book_id == books.c.id,
            )
        ).scalar_subquery()
    )

##### Composing from Column Properties at Mapping Time

It is possible to create _mappings_ that __combine multiple `ColumnProperty` objects__ together. The `ColumnProperty` will be interpreted as a SQL expression when used in a _Core expression context_, provided that it is targeted by an existing expression object; this works by the _Core_ detecting that the object has a `__clause_element__()` method which returns a SQL expression. However, if the `ColumnProperty` is used as a _lead object_ in an expression where there is __no other Core SQL expression object to target it__, the `ColumnProperty.expression` attribute will __return the underlying SQL expression__ so that it can be used to _build SQL expressions consistently_. Below, the `File` class contains an attribute `File.path` that _concatenates a string token_ to the `File.filename` attribute, which is itself a `ColumnProperty`.

In [13]:
class File(Base):
    __tablename__ = "files"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(64))
    extension = Column(String(8))
    filename = column_property(name + "." + extension)
    path = column_property("C:/" + filename.expression)

When the `File` class is used in expressions normally, the attributes assigned to _filename_ and _path_ are __usable directly__. The use of the `ColumnProperty.expression` attribute is __only necessary when using the `ColumnProperty` directly__ within the mapping definition.

```
q = session.query(File.path).filter(File.filename == "foo.txt")
```